# Nightlight Satellite Image Export
## Prerequisites
### Input

* CSV with 4 columns:
    * ID
    * year of survey
    * Latitude-Coordinates
    * Longitude-Coordinates

### Execution

1. create a pandas dataframe with all the information and delete all unnecessary entries (in our case it is all entries who are from 2011 or earlier)
2. creation of a range of year in string format (yyyy-mm-dd) instead a year in integer format
3. create a Featurecollection with all the information needed to get the correct images (Area of interest with a radius of 5.5 km)
4. create an image with the Featurecollection and the median
5. export them to google drive (there is no need for Authentication, it gets exported to the same account used for google earth engine)
6. create a function which downloads all images instantly as long as there are not the same amount of images in the drive compared to the length of the dataframe.

In [1]:
import ee
import pandas as pd
from typing import Tuple
import utils
import geemap #Nur für Visualisierungzwecke im jetzigen Zustand, wird beim Export der Daten wieder entfernt

In [2]:
#ee.Authenticate()
ee.Initialize()

In [30]:
#CONSTANTS
#Path of CSV-File with columns ID,year,LATITUDE,LONGITUDE
csvpath = '../dataResearch/firstSample.csv'
#name all column names
#year
year = 'year'
#Latitude and Longitude Coordinates
LATNUM = 'LATNUM'
LONGNUM= 'LONGNUM'
#ID for Filenames
surveyid = 'ID'
#Export parameters
DHS_EXPORT_FOLDER = 'dhs_tfrecords_raw'
# image export parameters
PROJECTION = 'EPSG:3857'  # see https://epsg.io/3857
#1100 pixel times 10m equals 11000mx11000m image
SCALE = 10                # export resolution: 10m/px
IMAGE_DIMENSION = '1100'  #WIDTHxHEIGHT image dimension = 1100px * 10m = 11000m = 11km

In [33]:
#create Dataframe with pandas
df = pd.read_csv(csvpath)
#create Dataframe where year is 2012 or higher
df = df[df.year >= 2012]
for i in range(5):
    start_date,end_date = utils.surveyyear_to_range(df[year].iloc[i],satellitename='nl')
    # get the VIIRS image collection, we're using the "avg_rad" band
    dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(start_date, end_date))
    nighttime = dataset.select('avg_rad').median()
    # initialize map on AOI (here Sao Paolo)
    lat = float(df[LATNUM].iloc[i].replace(',', '.'))
    lon = float(df[LONGNUM].iloc[i].replace(',', '.'))
    aoi = ee.Geometry.Point(lon,lat)
    name = df[surveyid].iloc[i]
    task = ee.batch.Export.image.toDrive(image=nighttime,region=aoi,folder=DHS_EXPORT_FOLDER,fileNamePrefix=name,scale=SCALE,description=name)
    task.start()

TypeError: surveyyear_to_range() got an unexpected keyword argument 'satellitename'

## Unten nur zur Sicherstellung Code, der die Images anzeigt:

In [ ]:
nighttimeVis = {min: 0.0, max: 60.0}
Map = geemap.Map()
Map.setCenter(lon, lat, 9)
Map.addLayer(nighttime, nighttimeVis, 'Nighttime')
Map